# Additive Decomposition

## Overview

<style>
    table.custom-table {
        max-width: 1000px;
        width: 100%;
        margin: 0 auto; /* centers the table on the page */
    }

    table.custom-table td {
        background-color: #fff;
    }
    
    table.custom-table th, table.custom-table td {
        text-align: center;
        vertical-align: middle;
        padding: 5px;
        width: 500px; /* distribute the total width equally among three columns */
    }

    table.custom-table img {
        width: 100%;
        display: block; /* removes any gap under the image */
    }
</style>


<table class="custom-table">
    <thead>
        <tr>
            <th>Output: Time Series Decomposition (Additive)</th>
            <th>Input: Original Time Series</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>
                <img src="src/02_output.png">
            </td>
            <td>
                <img src="src/02_input.png">
            </td>
        </tr>
    </tbody>
</table>

## Data

In [4]:
import pandas as pd

df_passenger = pd.read_csv('../data/airline-passengers.csv', parse_dates=['Month'], index_col='Month')
df_passenger = df_passenger.asfreq('MS')
df_passenger

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


## Individual Components

- Trend (S)
- Seasonality (S)
- Residual or Irregular Component (I)

In [5]:
import statsmodels.api as sm 

data = df_passenger['Passengers'].values
result = sm.tsa.seasonal_decompose(data, model='additive', period=12)

df_component = (df_passenger
 .assign(
    trend = result.trend,
    seasonal = result.seasonal,
    residual = result.resid)
 .dropna())

df_component

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,63.830808,-42.622475
1949-08-01,148,127.250000,62.823232,-42.073232
1949-09-01,136,127.958333,16.520202,-8.478535
1949-10-01,119,128.583333,-20.642677,11.059343
1949-11-01,104,129.000000,-53.593434,28.593434
...,...,...,...,...
1960-02-01,391,461.375000,-36.188131,-34.186869
1960-03-01,419,465.208333,-2.241162,-43.967172
1960-04-01,461,469.333333,-8.036616,-0.296717


In [6]:
df_component['component_sum'] = df_component.trend + df_component.seasonal + df_component.residual
df_component

,Passengers,trend,seasonal,residual,component_sum
Month,,,,,
1949-07-01,148,126.791667,63.830808,-42.622475,148.0
1949-08-01,148,127.250000,62.823232,-42.073232,148.0
1949-09-01,136,127.958333,16.520202,-8.478535,136.0
1949-10-01,119,128.583333,-20.642677,11.059343,119.0
1949-11-01,104,129.000000,-53.593434,28.593434,104.0
...,...,...,...,...,...
1960-02-01,391,461.375000,-36.188131,-34.186869,391.0
1960-03-01,419,465.208333,-2.241162,-43.967172,419.0
1960-04-01,461,469.333333,-8.036616,-0.296717,461.0


In [7]:
dff = df_component.drop(columns=['Passengers','component_sum'])

## Additive Model

$y_t = T_t + S_t + e_t$

In [8]:
import plotly.express as px

In [9]:
px.area(df_component, x=df_component.index, y=['Passengers'], title='Decomposition of Airline Passengers Data', width=800, height=400)

In [10]:
px.area(df_component, x=df_component.index, y=['trend','seasonal','residual'], width=800, height=400)

In [11]:
dff = dff.melt(ignore_index=False)

In [12]:
px.area(dff, x=dff.index, y=['value'], facet_col='variable', width=800, height=400)

In [13]:
df_component[['residual']].describe().T[['mean', 'std']]

,mean,std
residual,-0.751263,19.340535
